In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, BatchNormalization, Bidirectional
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:

# Load IMDB dataset
top_words = 10000  # Number of words to consider
maxlen = 200  # Maximum review length
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)


In [ ]:

# Padding sequences to ensure uniform input size
x_train = pad_sequences(x_train, maxlen=maxlen, padding='post')
x_test = pad_sequences(x_test, maxlen=maxlen, padding='post')


In [ ]:

# Build LSTM Model
model = Sequential([
    Input(shape=(maxlen,)),
    Embedding(input_dim=top_words, output_dim=128),
    Bidirectional(LSTM(64, activation='tanh', return_sequences=True)),
    BatchNormalization(),
    Dropout(0.3),
    LSTM(32, activation='tanh'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:

# Train the model
history = model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}, Test Loss: {loss:.4f}")


In [ ]:

# Plot accuracy
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()


In [ ]:

# Confusion Matrix
y_pred = (model.predict(x_test) > 0.5).astype('int32')
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:

# Classification Report
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

# Save the model
model.save('imdb_lstm_model.h5')